In [56]:
import math
from collections import Counter
import pickle
import re
import nltk
from nltk.corpus import stopwords
from lyricsgenius import Genius

with open('../lyrics-updated.pkl', 'rb') as f:
    song_lyrics = pickle.load(f)


with open('../classifications.pkl', 'rb') as f1:
    classifications = pickle.load(f1)

with open('../big_songs.pkl', 'rb') as f2:
    big_songs = pickle.load(f2)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def tokenize(lyrics):
    words = lyrics.lower().split()
    return [word for word in words if word not in stop_words]

k1 = 1.0
b = 0.5

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\civip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
# call the api to get the lyrics

genius = Genius('HhcPCGMwENzdfMQlauEV1Om0UormkWc6Wrwxri-LFTfBSNkzhwMSVUnR5tZG8eXW')
genius.remove_section_headers = True
genius.skip_non_songs = False
genius.excluded_terms = ["Remix", "Live"]

# song_name = "Happy"
# artist = "Pharrel Williams"
# song_name = "Firework"
# artist = "Katy Perry"
song_name = "I Hate Everything About You"
artist = "Three Days Grace"

song = genius.search_song(song_name, artist)
pos = song.lyrics.find(f'{song_name} Lyrics')
query = song.lyrics[pos:]



Searching for "I Hate Everything About You" by Three Days Grace...
Done.


In [58]:
# then remove  capital letters and non alphanumeric characters
query = query.lower()
query = ''.join([char if char.isalpha() or char.isspace() else '' for char in query.replace('\n', ' ')])
# it might seem crazy what im bout to say
query



'i hate everything about you lyrics every time we lie awake after every hit we take every feeling that i get but i havent missed you yet every roommate kept awake by every sigh and scream we make all the feelings that i get but i still dont miss you yet  only when i stop to think about it  i hate everything about you why do i love you i hate everything about you why do i love you every time we lie awake after every hit we take every feeling that i get but i havent missed you yet  only when i stop to think about it you might also like i hate everything about you why do i love you i hate everything about you why do i love you  only when i stop to think about you i know only when you stop to think about me do you know  i hate everything about you why do i love you you hate everything about me why do you love me i hate you hate i hate you love me i hate everything about you why do i love youembed'

In [59]:
# tokenize all documents
tokenized_songs = {}
for category, all_lyrics in big_songs.items():
    tokenized_songs[category] = tokenize(all_lyrics)

# calculate document lengths
doc_lengths = {}
for  category, all_lyrics in tokenized_songs.items():
    doc_lengths[category] = len(all_lyrics)

# avg doc length
avg_doc_length = sum(doc_lengths.values()) / len(doc_lengths)


In [60]:
# calculate doc freqs
doc_freqs = {}
for doc in tokenized_songs.values():
    unique_terms = set(doc)
    for term in unique_terms:
        doc_freqs[term] = doc_freqs.get(term, 0) + 1

In [61]:
# bm25 algorithm
def bm25(query, document, doc_length):
    score = 0.0
    query_terms = tokenize(query)
    term_freqs = Counter(document)
    
    for term in query_terms:
        if term in doc_freqs:  # term must appear in the corpus
            # Calculate BM25 components
            doc_freq = doc_freqs[term]
            idf = math.log((len(big_songs) - doc_freq + 0.5) / (doc_freq + 0.5) + 1)
            term_freq = term_freqs[term]
            term_score = idf * (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * doc_length / avg_doc_length))
            score += term_score
    return score

In [62]:
scores = []
for doc_id, doc in tokenized_songs.items():
    score = bm25(query, doc, doc_lengths[doc_id])
    scores.append((doc_id, score))

# Sort documents by score in descending order
ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)

# Display the ranking results
print("Document Rankings:")
for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
    print(f"Rank {rank}: Document {doc_id} with score {score:.4f}")

Document Rankings:
Rank 1: Document uplifting with score 70.0821
Rank 2: Document dark with score 69.3581
Rank 3: Document energetic with score 66.3306
Rank 4: Document dynamic with score 60.0821
Rank 5: Document anger with score 59.5034
Rank 6: Document fierce with score 58.2192
Rank 7: Document cynical with score 6.2137
